# 第8章 員が構造をデータから推測する因果探索

## 8.1 因果探索の概要

### 8.1.1 因果探索の位置づけ

[第2章](../chap1_4/chap2.ipynb#25-因果効果推定のためのグラフ)ではDAGを用いて因果を考えて来たが、事前知識がな、系が複雑などDAGが描けない場合、因果探索を行う。

因果探索では、因果の方向と強さの推定を行う。

![図8.3](images/chap8/image.png)

実世界の活用例

Amazonのプライムデーで1時間の障害(2018年)。最大1億ドルの売上損失が発生した可能性。因果探索を使えば障害の原因をすぐに特定、もしくは防止できる。

米国では因果探索を用いてCOVID-19のワクチン接種をためらう要因を分析し対策が取られていた。

### 8.1.2 グラフィカルモデルとベイジアンネットワーク

員が探索ではグラフィカルモデルが欠かせない。

因果探索で用いるグラフィカルモデル

* 有向グラフ

  * ベイジアンネットワーク

    状態A、Bがあるときに、AからBに遷移する確率を示す。因果関係ではないので別途検証する必要があることに注意

    * 静的: 時系列を考慮しない([8.2節](#82-因果探索手法の全体像))
    * 動的: 時系列を考慮する([8.4節](#84-時系列を伴う因果探索の全体像))
* 無向グラフ

  * ボルツマンマシン(Boltzman Machine)

ベイジアンネットワークの定義

* $|X_1, X_2 \ldots X_n|$: ノードの集合$V$
* $E$: エッジの集合
* $parent(X_i)$: $X_i$親ノード

ベイジアンネットワークは双方向矢線のない有向非巡回グラフ$G=(V, E)$なので、次のように表現する

$$
P(X_1, X_2 \ldots X_n) = \prod_{i=1}^n P(X_i | parent(X_i))
$$

つまり、同時分布(左項)が親ノードで条件付けられた確率の積(右項)として記述できる

ベイジアンネットワークの特性として、親と子の依存関係を各要素に分解できる分解特性がある

例: 雲ができた後に地面に水たまりができる確率

* $P(A)$: 雲が発生する確率
* $P(B|A)$: 雲が発生したときに雨が降る確率
* $P(C|B)$: 雨が降ったあとに水たまりができる確率

$水たまりができる確率 = P(C|B)P(B|A)P(A)$

### 8.1.3 ベイジアンネットワークの特徴

グラフィカルモデルを用いることによる、各変数の依存関係や構造に関する解釈性と説明可能性があるため、原因の深堀りや構造化が可能

### 8.1.4 ベイジアンネットワークから因果ダイアグラムへ

ベイジアンネットワークは変数の依存関係を示すが因果関係は含まない。そこから因果関係を含むグラフを作成するには以下を用いる。

![図8.9](images/chap8/image-1.png)

1. 因果モデル

   決定論的な関数方程式で因果関係を表現する

   * $f_i$: 任意の関数
   * $\epsilon_i$

   $$
   X_i = f_i (parent(X_i), \varepsilon_i) \ (i=1,\ldots, n)
   $$
2. 因果ダイアグラム

   因果モデルをもとに、親ノード($parent(X_i)$)から子ノード($X_i$)への矢印を引きグラフを描いたもの

   * ノード: 対象の確率変数を表す
   * エッジの方向: 直接的に影響を与えうる関係性
     ![図8.10](images/chap8/image-2.png)

このような因果ダイアグラムを作る際などに、相関や依存関係から因果関係を推定するには、以下の因果仮定(Causal assumptions)を置いて因果探索を行う必要がある。

1. 因果マルコフ条件(Causal Markov Condition)

   ノード$X_i \in X$は、親ノードで条件付けられた場合、それが直接間接的にかかわらず、非子孫の他のノード$X_i \in X$と条件付き独立。言い換えると、親ノード経由でのみたどり着けるノード(エッジ方向は無視)とは条件付き独立ということ

   この条件によって推論の対象範囲を絞り込める
2. 忠実性(Faithfulness)

   任意のノード間において、d分離以外には統計的独立関係が成り立たないこと

   たとえば、ノード$X_1, X_2, X_3$があるときに

   * $X_2 \to X_3$
   * $X_2 \leftarrow X_1\to X_3$

   という関係があるとする図で示すと下図となる
   ![図8.12](images/chap8/image-3.png)

   $X_2 \to X_3$の影響が、$X_1$からの影響で相殺され、$X_2, X_3$が独立している場合、d分離以外の独立関係があるため忠実性は満たされない。

   式で表現すると以下になる。

   $$
   X_2 = \alpha X_1 + \varepsilon_1
   X_3 = \beta X_1 + \gamma X_2 ; \varepsilon_2
   $$

   ここで、$\beta = -\alpha\gamma$であるとき、

   $$
   \beta X_1 + \gamma X_2 \approx 0
   $$

   となり相殺が成り立つ

   d分離おさらい

   * 直接的な因果関係: パスが連鎖(chain)構造(X→M→Y)がある場合、Mが集合Zに含まれるとき、XとYは独立
   * 共通原因: パスが分岐(fork)構造(X←M→Y)がある場合、Mが集合Zに含まれるとき、XとYは独立
   * 共通効果：パスが合流(collider)構造(X→M←Y)がある場合、MもMの子孫も集合Zに含まれない限り、XとYは独立
3. 因果的十分性(Causal Sufficiency): 未観測の交絡因子が存在しないこと
4. 非巡回性(Acyclicity): DAGの定義に含まれる

## 8.2 因果探索手法の全体像

因果探索のアプローチは以下の4つに分けられる

1. 制約ベース(Constraint based)
2. スコアベース(Score based)
3. 関数因果モデル(Function causal model)
4. 勾配ベースモデル(Gradient based model)

### 8.2.1 制約ベース

構造学習

* 変数間の条件付き独立性
* その条件付き独立性と非巡回の仮定を満たす因果構造から因果関係を導き出す

代表的なアルゴリズム: PCアルゴリズム

![図8.14](images/chap8/image-4.png)

#### Step 1 必要なつながりを残す

1. すべてのノード間が無向エッジで繋がれたグラフを用意する
2. 2変数が互いに独立、または他の変数集合で条件付けたときに独立となる場合、エッジを除外。具体的には、$\chi$ニ乗検定で独立性が確認できればエッジを削除
3. まだ隣接しているすべての変数の組に対して、1つの変
   数で条件付けて独立性を検定
4. このプロセスがすべての変数のペアで実施された段階で終了

#### Step 2 矢印の向きの設定

5. エッジ$A-B-D$が存在し、エッジ$A-D$が存在しない状況において、$B$で条件付けたときに$A$と$D$が従属になる場合、$A \to B \larr D$となる
6. 最後に、$B-C$の関係はAとCが従属かつBで条件付けたときにAとCは独立(3.より)であることから、$B \to C$のように方向付けられる。

   逆に、$A \rarr B \larr C$としてしまうと3.($A \perp C|B$)と矛盾する

注意

* 通常は複数の因果グラフの候補から絞り込みを行う。ここで、同じ条件付き独立性を与える因果グラフの集合をマルコフ同値類(Markov equivalence class)というが、この中から1つの因果グラフに絞りきれない場合がある
* サンプルサイズが小さい場合、$\chi$ニ乗分布を用いた近似の精度が悪くなる
* PCアルゴリズムは条件付き独立の検定を実施する順番に影響されやすく、高次のデータを用いた場合は、推定結果が不正確。これに対して、PC-stableというアルゴリズムで対応可能
* PCアルゴリズムでは因果的十分性が仮定されているため、未観測の交絡因子や選択バイアスがある場合には、因果探索の推定精度が下がる
* $\chi$二乗検定の有意水準($\alpha=0.05$など)は、明確に決定できないハイパーパラメータであり、設定しだいで検定結果が変わる
* すべての変数の組合せに対して統計的検定などの処理を行うため、変数の数に対して指数関数的に計算量が増加し処理負荷がかかる

### 8.2.2 スコアベース

モデルの精度を評価するスコアを設定し、因果グラフの集合から最もよいグラフを探す

代表的なアルゴリズム: Greedy Equivalence Search(GES)

![図8.15](images/chap8/image-5.png)

## GESアルゴリズム

1. エッジのないグラフを作成
2. 有向エッジを追加
3. 不要なエッジを削除

2.と3.を繰り返しながら、ベイズ情報量規準(BIC)などのスコアを計測し、改善されなくなるまで続ける。

### 8.2.3 関数因果モデルに基づく因果探索

構造的因果モデルともいう

制約ベースの手法における独立性の検定に関連する課題([8.2.1項](#821-制約ベース))。特に以下の大きな課題に対応する。

* すべての変数間の因果の方向が定まらない
* 適切な条件付き独立性テストを行うには、大きなサンプルサイズが必要

代表的な手法: LiNGAM(Linear Non-Gaussian Acyclic Model)。非ガウス性(ガウス分布に従わない性質)を使用する。

仮定

1. 誤差変数は非ガウス分布

   因果の方向をデータの分布から特定するために必要

   具体的に、XとYの例で考える。このとき因果方向は$X \to Y$(XをYに回帰する場合), $X \gets Y$(YをXに回帰する場合)の2通り考えられる。

   両方に対し、誤差分布をガウス分布と連続一様分布の2パターン設定してプロットした結果が下図である。

   ![図8.16](images/chap8/image-6.png)

   ガウス分布のパターンの比較では、残差$\hat{\varepsilon}_Y, \hat{\varepsilon}_X$の分布は同じだが、下段の連続一様分布では$X \gets Y$(右図)の残差$\hat{\varepsilon}_X$の分布がYの値によって変化している。

   このように、連続一様分布(もしくは他の非ガウス分布)で非対称性が現れるため、因果方向を特定可能。
2. 線形の構造方程式モデル
3. 非巡回性(これまでの手法と同様にDAGであること)
4. 隠れた共通原因なし(No hidden common causes: 2つ以上の観測変数に影響を与える未観測変数が存在しないこと)

LiNGAMの実行手順
#### Step 1 セミパラメトリックなSEMの構築
LiNGAMは線形モデル(パラメトリック)と非ガウス性の誤
差変数(ノンパラメトリック)を使用しているため、セミパラメトリックモデルという。

構造方程式モデル(SEM)を構築する際、上記のLiNGAMの仮定を満たす必要がある。

線形の構造方程式モデル
$$
X_i = \sum_{j \neq i} B_{ij} X_j + \varepsilon_i \;\; (i = 1, \ldots, n)
$$
* $B_{ij}$: X_jからX_iへの因果効果を表す係数
* $\varepsilon_i$: 誤差項

上式を行列を用いて表すと、
$$
X = BX + \varepsilon
$$

メモ: 非ガウス性を評価する方法
1. データの可視化: ヒストグラム、散布図
2. 尖度と歪度の計算: ガウス分布の場合は尖度3、歪度0
3. 統計的検定: シャピロ・ウイルク検定、コルモゴロフ・スミルノフ検定、リリフォース検定、ジャーク・ベラ検定など

#### Step 2 独立成分分析(Independent Component Analysis: ICA)

非ガウス性を利用して独立成分を抽出する方法。

例えば、様々な楽器の音源を混ぜた混合音の信号をn次元ベクトルの信号とすると

$$
X(t) = AS(t)
$$
* $X(t) = (X_1(t), X_2(t), \ldots, X_n(t))^T$: 観測データ(信号)
* A: 混合行列(各音源がどのように混ざり合っているかを示す)
* S: 独立成分(独立した各音源のベクトル。これを導き出したい)

![図8.17](images/chap8/image-7.png)

ここで、観測データに対し以下の分離モデルを考える
$$
u(t) = WX(t)
$$
* $U(T)$ = (u_1(t), u_2(t), \ldots, u_n(t))^T: 分離信号
* W: 復元行列

ICAアルゴリズムは、観測データ(混合信号)を用いて、復元行列$W$を更新しながら、分離信号u(t)を、統計的に独立となるように生成する方法。目的関数は非ガウス性が最大になること。

LiNGAMでは、上記の2式をXについて整理し、以下の式の形で表す
$$
(I - B)X = \varepsilon \\
W = I - B
$$
* I: 単位行列
* B: $X_j$から$X_i$絵hの因果効果を表す係数の行列

メモ: ICAアルゴリズムのの具体的な実行手順
1. データ前処理
   
   ICAを適用する前に、データを中心化(平均を原点に移動)および白色化(相関を除去し、各成分の分散が1になるように正規化)
   
2. 混合行列の更新、独立成分の最大化
   
   分離信号$u(t)$の独立性の代表的な指標として相互情報量$I(u) \ (u \gt 0)$があり、$I(u) = 0$のとき統計的に独立とみなす。したがって、$I(u)$が最小になるように復元行列$W$を更新していく
   
3. 収束判定
   
   更新による復元行列$W$の変動がしきい値を下回るまで、独立性を最大化する計算を繰り返す

4. 独立成分の抽出
   
   収束した復元行列$W$を用いて、元のデータから独立成分を抽出

#### Step 3 因果的順序の定義

ICAから得られた変数について、因果的順序を決定する。

この順序に従って変数を並び替えると、係数行列が下三角行列になる。この特性を利用して因果的順序を推定します。

つまり、

$$
X_{並び替え後} = B_{並び替え後} X_{並び替え後} + \varepsilon_{並び替え後}
$$

の式が成り立つような厳密に下三角行列$B$を求める。

#### Step 4 因果関係・因果効果の推定

最後に、$B_{並び替え後}$が変数間の関係性と因果効果を表します。

他のアルゴリズムとしては、
* Direct LiNGAM
* 未観測の共通原因を考慮したParceLiNGAM、RCD
* 時系列を考慮したVAR-LiNGAM(後述)

特に、Direct LiNGAMは、ICA-LINGAMと比較して計算量が多いが、固定数のステップで収束することが保証されている。

### 8.2.4 勾配ベース

### 8.2.5 精度検証

## 8.3 因果探索の手法の実装

## 8.4 時系列を伴う因果探索の全体像

### 8.4.1 時系列を伴う因果探索の概要

### 8.4.2 制約ベース：PCMCIの概要

### 8.4.3 関数因果モデル：VAR-LiNGAMの概要

### 8.4.4 その他のモデルの概要

## 8.5 時系列を伴う因果探索の実装

## 8.6 因果探索の課題

おわりに
